# Copycat

In reading about GPT-3 I came across Melanie Mitchell's article ["Can GPT-3 Make Analogies?"](https://medium.com/@melaniemitchell.me/can-gpt-3-make-analogies-16436605c446). Both this article and [its follow-up](https://medium.com/@melaniemitchell.me/follow-up-to-can-gpt-3-make-analogies-b202204bd292) were written in August of 2020 and it seems GPT-3 has changed quite a bit since then. Most notably GPT-3 now offers four different models, each with their own tradeoffs between speed, cost, and capabilities. The most capable (and most expensive) model, `text-da-vinci-002`, uses training data from as recently as June of 2021. 

This made me curious: 
  1. With the changes to GPT-3, has its ability to solve letter-string analogy problems improved? 
  2. How does model selection affect performance?
  3. How can fine-tuning be used to increase performance?
  
I'll start by feeding GPT-3 the same prompts Dr. Mitchell did (from the original article as well as its follow-up) using `text-da-vinci-002` through the OpenAI Python API. After that I'll try the other models and compare the results.

If you'd like to run or modify this code you'll just need to get an OpenAI API key and set your OPENAI_API_KEY env variable to it. I'll try to keep track of the overall cost in credits as I go. Additionally, I've written some helper functions that will be kept in `gpt_helpers.py` so I don't clutter this notebook. 

In [1]:
from gpt_helpers import get_completion

<OpenAIObject text_completion id=cmpl-5vNG1BTLGm721EOBXfCAHXnzyjUtN at 0x10d7a3a60> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\np q d"
    }
  ],
  "created": 1664355781,
  "id": "cmpl-5vNG1BTLGm721EOBXfCAHXnzyjUtN",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 5,
    "prompt_tokens": 19,
    "total_tokens": 24
  }
}